# SVM

In [1]:
import os
import os.path
import numpy as np
import time
from tqdm import tqdm

assert os.path.exists('data')

In [2]:
def load_dataset(data_source_filename):
  print("Loading %s... " % data_source_filename.split("/")[-1], end="")
  t = time.time()
  dataset = dict()
  with np.load(data_source_filename, allow_pickle=True) as source_file:
    for key in source_file.keys():
      # print(key)
      dataset[key] = source_file[key].tolist()
  print("done (%.1fs)" % (time.time()-t), flush=True)
  return dataset

data_source_filenames = [os.path.join('data', fn) for fn in os.listdir('data')
                            if os.path.isfile(os.path.join('data', fn)) and fn[-3:]=='npz']
data_source_filenames

['data/StClare_facs_danish.npz',
 'data/SDHK_Latin.npz',
 'data/StClare_facs_latin.npz',
 'data/StClare_dipl_danish.npz',
 'data/StClare_dipl_latin.npz',
 'data/velux_facs_danish.npz',
 'data/SDHK_Swedish.npz',
 'data/Colonia.npz',
 'data/SemEval2015.npz',
 'data/velux_dipl_latin.npz',
 'data/velux_facs_latin.npz',
 'data/velux_dipl_danish.npz']

In [3]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

bin_width = 25
n_features_max = 1000

results = dict()

In [4]:
for data_source_filename in set(data_source_filenames).symmetric_difference(set(results.keys())):
#for data_source_filename in data_source_filenames:
    dataset = load_dataset(data_source_filename)
    y = np.asarray([np.mean(item['date']) if type(item['date']) is tuple else item['date'] for item in dataset['data']], np.int)
    y = bin_width*(y//bin_width)
    print(" Classes in data: %s" % np.unique(y))
    
    training_set = dataset['folds']['train']
    validation_set = dataset['folds']['val']
    test_set = dataset['folds']['test']
    training_validation_set = list()
    training_validation_set.extend(training_set)
    training_validation_set.extend(validation_set)
    
    results[data_source_filename] = dict()
    print(" Running estimations", flush=True)
    for feature_set in list(dataset['feature_sets'].keys()):
        X = dataset['feature_sets'][feature_set]
        print("  Loaded feature set %s" % feature_set)
        n_samples, n_features = X.shape
        y_pred = np.zeros(y.shape)
        
        # Chi^2 feature selection
        if n_features > n_features_max:
            print("   Reducing feature space")
            feature_selector = SelectKBest(chi2, k=n_features_max).fit(X[training_validation_set], y[training_validation_set])
            X = feature_selector.transform(X)
            n_features = X.shape[1]
        # Zero mean
        X = X.tocsc(copy=True).astype(np.float)
        for i in range(X.shape[1]):
            if X.indptr[i] < X.indptr[i+1]:
                X.data[X.indptr[i]:X.indptr[i+1]] -= np.mean(X.data[X.indptr[i]:X.indptr[i+1]])
        X = StandardScaler(with_mean=False).fit_transform(X)
        X = X.tocsr(copy=True)

        C = np.logspace(-10, 5, 20)
        validation_accuracy = list()
        print(" Making the estimator", flush=True)
        for c in tqdm(C, desc="  Searching for regularisation parameter C"):
            estimator = LinearSVC(dual=(n_samples<=n_features), class_weight='balanced', C=c)
            estimator.fit(X[training_set, :], y[training_set])
            validation_accuracy.append(accuracy_score(y[validation_set], estimator.predict(X[validation_set, :]).ravel()))
        estimator = LinearSVC(dual=(n_samples<=n_features), 
                              class_weight='balanced', C=C[np.argmax(validation_accuracy)])
        print("  Best C:", C[np.argmax(validation_accuracy)])
        estimator.fit(X[training_validation_set, :], y[training_validation_set])
        y_pred = estimator.predict(X[test_set, :])
        results[data_source_filename][feature_set] = {'accuracy': accuracy_score(y[test_set], y_pred), 
                                                      'y_pred': y_pred, 'y_true': y[test_set]}

    print()
    print(" Results")
    for feature_set in results[data_source_filename].keys():
        print("  %s, %.1f%%" % (feature_set, 100*results[data_source_filename][feature_set]['accuracy']))
        print("  unique predicted classes: %s" % np.unique(results[data_source_filename][feature_set]['y_pred']))

    def class_counter(data):
        count = dict()
        for e in data:
            if e in count:
                count[e] += 1
            else:
                count[e] = 1
        return count

    count = class_counter(y)
    print(" Controls")
    print("  MLE choice: %.1f%%" % (100*np.max(list(count.values()))/len(y)))
    print("  Uniform: %.1f%%" % (100*np.mean([np.sum(np.random.choice(np.unique(y), size=len(y), replace=True)==y)/len(y) for i in range(1000)])))
    print("  Weighted: %.1f%%" % (100*np.mean([np.sum(np.random.choice(y, size=len(y), replace=False)==y)/len(y) for i in range(1000)])))

Loading velux_dipl_latin.npz... done (0.1s)
 Classes in data: [1250 1275 1300 1325 1350 1375 1400 1425 1450 1475 1500 1525]
 Running estimations
  Loaded feature set bow_words
   Reducing feature space
 Making the estimator


  Searching for regularisation parameter C:   0%|          | 0/20 [00:00<?, ?it/s]/home/fredrik/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
  Searching for regularisation parameter C: 100%|██████████| 20/20 [00:00<00:00, 29.11it/s] 

  Best C: 6.158482110660255e-10
  Loaded feature set tfidf_words
   Reducing feature space
 Making the estimator



  Searching for regularisation parameter C:   0%|          | 0/20 [00:00<?, ?it/s]/home/fredrik/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
  Searching for regularisation parameter C: 100%|██████████| 20/20 [00:00<00:00, 40.48it/s]

  Best C: 0.007847599703514624
  Loaded feature set character_ngram_1
 Making the estimator


...

  Best C: 0.00020691380811147902
  Loaded feature set word_ngram_2
   Reducing feature space
 Making the estimator


  Searching for regularisation parameter C:  50%|█████     | 10/20 [00:00<00:00, 89.23it/s]/home/fredrik/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
  Searching for regularisation parameter C: 100%|██████████| 20/20 [00:16<00:00,  1.25it/s]


  Best C: 0.0012742749857031347
  Loaded feature set word_ngram_3
   Reducing feature space
 Making the estimator


  Searching for regularisation parameter C:  60%|██████    | 12/20 [00:00<00:00, 29.05it/s]/home/fredrik/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
  Searching for regularisation parameter C: 100%|██████████| 20/20 [00:05<00:00,  3.54it/s]


  Best C: 69.5192796177562

 Results
  bow_words, 25.5%
  unique predicted classes: [1700]
  tfidf_words, 29.7%
  unique predicted classes: [1700 1725 1775 1800 1825 1850 1875 1900 1925 1950 1975 2000]
  character_ngram_1, 31.0%
  unique predicted classes: [1700 1725 1775 1800 1825 1850 1875 1900 1925 1950 1975 2000]
  character_ngram_2, 32.9%
  unique predicted classes: [1700 1725 1775 1800 1825 1850 1875 1900 1925 1950 1975 2000]
  character_ngram_3, 35.5%
  unique predicted classes: [1700 1725 1775 1800 1825 1850 1875 1900 1925 1950 1975 2000]
  word_ngram_1, 32.9%
  unique predicted classes: [1700 1725 1775 1800 1825 1850 1875 1900 1925 1950 1975 2000]
  word_ngram_2, 27.2%
  unique predicted classes: [1700 1725 1775 1800 1825 1850 1875 1900 1925 1950 1975 2000]
  word_ngram_3, 27.4%
  unique predicted classes: [1700 1725 1800 1825 1850 1875 1900 1925 1950 1975 2000]
 Controls
  MLE choice: 25.5%
  Uniform: 8.3%
  Weighted: 11.9%


In [5]:
for data_source_filename in results.keys():
    dataset = load_dataset(data_source_filename)
    print(" Saving %s... " % data_source_filename.split("/")[-1], end="")
    t = time.time()
    if 'linearsvc' in dataset.keys():
        dataset.pop('linearsvc')
    np.savez_compressed(data_source_filename, **dataset, linearsvc=results[data_source_filename])
    print("done (%.1fs)" % (time.time()-t), flush=True)

Loading velux_dipl_latin.npz... done (0.2s)
Saving velux_dipl_latin.npz... done (0.4s)
Loading StClare_dipl_danish.npz... done (0.0s)
Saving StClare_dipl_danish.npz... done (0.1s)
Loading velux_dipl_danish.npz... done (0.0s)
Saving velux_dipl_danish.npz... done (0.1s)
Loading velux_facs_danish.npz... done (0.0s)
Saving velux_facs_danish.npz... done (0.1s)
Loading StClare_dipl_latin.npz... done (0.1s)
Saving StClare_dipl_latin.npz... done (0.3s)
Loading SDHK_Latin.npz... done (1.8s)
Saving SDHK_Latin.npz... done (8.1s)
Loading StClare_facs_latin.npz... done (0.1s)
Saving StClare_facs_latin.npz... done (0.4s)
Loading StClare_facs_danish.npz... done (0.0s)
Saving StClare_facs_danish.npz... done (0.1s)
Loading velux_facs_latin.npz... done (0.1s)
Saving velux_facs_latin.npz... done (0.4s)
Loading Colonia.npz... done (7.4s)
Saving Colonia.npz... done (20.0s)
Loading SDHK_Swedish.npz... done (0.7s)
Saving SDHK_Swedish.npz... done (3.3s)
Loading SemEval2015.npz... done (0.2s)
Saving SemEval201